### Embeddings Extractor

In [24]:
import numpy as np
from torchvision.datasets import CIFAR100
from torchvision import transforms
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

You can swap the model for your prefered pretrained model.
I am using [SWAV](https://github.com/facebookresearch/swav), [paper](https://arxiv.org/pdf/2006.09882.pdf).

Which is unsupervised.

In [25]:
model = torch.hub.load('facebookresearch/swav', 'resnet50')
modules=list(model.children())[:-1]
model=nn.Sequential(*modules)
for p in model.parameters():
    p.requires_grad = False
model.cuda();

Using cache found in /home/dinari/.cache/torch/hub/facebookresearch_swav_master


In [26]:
tr_normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.228, 0.224, 0.225])
transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            tr_normalize])
dataset = CIFAR100('../data/cifar100/',train=True, download=True, transform=transforms)

Files already downloaded and verified


In [1]:
BATCH_SIZE = 8

In [14]:
loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

In [15]:
all_labels = []
all_embeddings = []
torch.cuda.empty_cache()
model.eval()
for data,labels in tqdm(loader):
    all_labels += labels.numpy().tolist()
    data = data.cuda()
    embeddings = model(data.cuda())
    all_embeddings.append(np.reshape(embeddings.detach().cpu().numpy(),(BATCH_SIZE,-1)))
all_embeddings = np.vstack(all_embeddings)

100%|██████████| 6250/6250 [04:35<00:00, 22.72it/s]


In [ ]:
DATA_PATH = '../data/cifar100/cifar100_embeddings.npy'
LABELS_PATH = '../data/cifar100/cifar100_labels.npy'

In [17]:
np.save(DATA_PATH,all_embeddings)
np.save(LABELS_PATH,np.array(all_labels))